# 범주형 데이터의 실수화  
1. 범주형 데이터의 실수화

1) 필요 라이브러리 import

In [34]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

2) 데이터셋 생성 및 확인

In [36]:
x_train = pd.DataFrame(["남성", "여성", "남성", "여성", "남성", "여성"], columns = ["성별"])

In [12]:
x_train.head(3)

,성별
0,남성
1,여성
2,남성


In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [14]:
x_train['성별'].value_counts()

남성    3
여성    3
Name: 성별, dtype: int64

3) 레이블 인코딩

In [15]:
encoding = LabelEncoder()
encoding.fit(x_train["성별"])
x_train["성별_인코딩"] = encoding.transform(x_train["성별"])

4) 인코딩 결과 확인

In [16]:
x_train

,성별,성별_인코딩
0,남성,0
1,여성,1
2,남성,0
3,여성,1
4,남성,0
5,여성,1


In [17]:
print(list(encoding.classes_))
print(list(encoding.inverse_transform([0,1])))

['남성', '여성']
['남성', '여성']


# 2. One-Hot-Encoding 

1) 데이터 확인

In [18]:
x_train.head(2)

,성별,성별_인코딩
0,남성,0
1,여성,1


2) One-Hot-Encoding

In [19]:
one_encoding = OneHotEncoder(sparse=False)
one_encoding.fit(x_train[["성별"]])
one_encoding.transform(x_train[["성별"]])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [37]:
print(one_encoding.get_feature_names())

['x0_남성' 'x0_여성']


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


3) 인코딩 결과 확인

In [21]:
x_train_one = pd.DataFrame(one_encoding.transform(x_train[['성별']]),
columns=['성별0', '성별1'])

In [22]:
x_train = pd.concat([x_train, x_train_one], axis=1)

In [23]:
x_train.head(3)

,성별,성별_인코딩,성별0,성별1
0,남성,0,1.0,0.0
1,여성,1,0.0,1.0
2,남성,0,1.0,0.0


# 텍스트 데이터 실수화

1. 단어의 빈도수를 카운트하여 단어를 수치화하는 방법

1) 필요 라이브러리 import

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

2) 데이터셋 생성 및 확인

In [25]:
corpus = [
            '청년 인재 개발 양성 과정',
            '인공지능 청년 양성',
            '미래 인공지능 데이터 대한민국',
            '데이터 원유 기술사 청년 개발',
]

In [26]:
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [27]:
type(corpus)

list

3) 텍스트 실수화 : 단어의 빈도수를 카운트

In [28]:
count_vect = CountVectorizer()
count_vect.fit(corpus)
count_vect.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [29]:
count_vect.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [30]:
features = count_vect.transform(corpus)

In [31]:
features

<4x11 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

4) 문서단어행렬로 변환 

In [32]:
vocab = count_vect.get_feature_names()

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
vocab

['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [39]:
DTM = pd.DataFrame(features.toarray(), columns = vocab).head()

In [40]:
DTM

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,1,1,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,1,0,1,0,1
2,0,0,0,1,1,1,0,0,1,0,0
3,1,0,1,0,1,0,0,1,0,0,1


In [41]:
count_vect.transform(['기술사 대한민국 인재 만세']).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

2 단어의 빈도수를 단어들마다 중요한 정도를 가중치로 주어 수치화하는 방법

1) 필요 라이브러리 import

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

2) 데이터set 생성 및 확인

In [44]:
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

3) 텍스트 실수화 : TF-IDF 방식

In [45]:
tfid = TfidfVectorizer()

In [46]:
tfid.fit(corpus)

TfidfVectorizer()

In [48]:
tfid.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [49]:
tfid.transform(corpus).toarray()

array([[0.41263976, 0.52338122, 0.        , 0.        , 0.        ,
        0.        , 0.41263976, 0.        , 0.        , 0.52338122,
        0.33406745],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.61366674, 0.        , 0.61366674, 0.        ,
        0.49681612],
       [0.        , 0.        , 0.        , 0.55528266, 0.43779123,
        0.55528266, 0.        , 0.        , 0.43779123, 0.        ,
        0.        ],
       [0.41263976, 0.        , 0.52338122, 0.        , 0.41263976,
        0.        , 0.        , 0.52338122, 0.        , 0.        ,
        0.33406745]])

In [50]:
features_idf = tfid.transform(corpus)

In [51]:
features_idf

<4x11 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

4) 문서단어행렬로 변환

In [53]:
vocab_idf = count_vect.get_feature_names()

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [56]:
DTM_idf = pd.DataFrame(features_idf.toarray(), columns = vocab_idf).head

In [57]:
DTM_idf

<bound method NDFrame.head of         개발        과정       기술사      대한민국       데이터        미래        양성  \
0  0.41264  0.523381  0.000000  0.000000  0.000000  0.000000  0.412640   
1  0.00000  0.000000  0.000000  0.000000  0.000000  0.000000  0.613667   
2  0.00000  0.000000  0.000000  0.555283  0.437791  0.555283  0.000000   
3  0.41264  0.000000  0.523381  0.000000  0.412640  0.000000  0.000000   

         원유      인공지능        인재        청년  
0  0.000000  0.000000  0.523381  0.334067  
1  0.000000  0.613667  0.000000  0.496816  
2  0.000000  0.437791  0.000000  0.000000  
3  0.523381  0.000000  0.000000  0.334067  >